# Code

The following code contains a `DT_Security` class which models a discrete-time security whose price evolves according to the Multi-period Binomial Model with parameters $S(0)$, $u$, $d$ and $r$.

A `DT_Option` class is also provided, which models European/American call/put options on the underlying discrete-time security.

In continuous time, the `CT_Security` and `CT_Option` classes assume a geometric Brownian motion model and options are priced by first constructing a discrete-time binomial approximation.

See below for example usage.

In [1]:
import numpy as np

class DT_Security:
    def __init__(self, S0, u, d, r):
        if u - d <=0:
            raise ValueError('u must be strictly greater than d')
        if 1 + r < d or 1 + r > u:
            raise ValueError('1 + r must lie between d and u')
            
        self.S0, self.u, self.d, self.r, = S0, u, d, r
        
    def price(self, t, i): # i = number of price increases, 0 <= i <= t
        if i < 0 or i > t:
            raise ValueError(
                'i (number of price increases) must lie between 0 and t'
            )
    
        return self.S0 * self.u**i * self.d**(t - i)
    
    def risk_neutral_p(self):
        return (1 + self.r - self.d)/(self.u - self.d)
    
    def random_price(self, t): # under risk-neutral probabilities
        X = np.random.binomial(t, self.risk_neutral_p())
        return self.S0*self.u**X*self.d**(t - X)
    
    def option(self, strike, maturity, put=False, American=False):
        return DT_Option(self, strike, maturity, put, American)
    
class DT_Option:
    def __init__(self, security, strike, maturity, put=False, American=False):
        self.security = security
        self.strike = strike
        self.maturity = maturity
        self.put = put
        self.American = American
        
    def payoff(self, t, i): # i = number of price increases, 0 <= i <= t
        S = self.security.price(t, i)
        K = self.strike
        if self.put:
            return max(K - S, 0)
    
        return max(S - K, 0)
    
    # 'prices' reutrns the list of possible prices at time t
    def prices(self, t):
        if t < 0 or t > self.maturity:
            raise ValueError('t must lie between 0 and maturity')
            
        if t == self.maturity:
            return [self.payoff(t, i) for i in range(t + 1)]
    
        p = self.security.risk_neutral_p()
        future_prices = self.prices(t + 1)
    
        def not_exercised(i):
            return (
                p*future_prices[i + 1] + (1 - p)*future_prices[i]
            )/(1 + self.security.r)
        
        def exercised(i):
            return self.payoff(t, i)
        
        def price(i):
            if self.American and exercised(i) > not_exercised(i):
                return exercised(i)
            return not_exercised(i)
        
        return [price(i) for i in range(t + 1)]
    
class CT_Security:
    def __init__(self, S0, volatility, r):
        self.S0, self.volatility, self.r = S0, volatility, r
        
    def drift(self): # under risk-neutral probabilities
        return self.r - self.volatility**2/2
        
    def random_price(self, t): # under risk-neutral probabilities
        X = np.random.normal(self.drift()*t, self.volatility*np.sqrt(t))
        return self.S0 * np.exp(X)
    
    def option(self, strike, maturity, put=False, American=False):
        return CT_Option(self, strike, maturity, put, American)

class CT_Option:
    def __init__(self, security, strike, maturity, put=False, American=False):
        self.security = security
        self.strike = strike
        self.maturity = maturity
        self.put = put
        self.American = American
        
    def initial_price(self, timesteps=1): # option price at time t = 0
        delta = self.maturity/timesteps
        step = self.security.volatility*np.sqrt(delta)
        
        dts = DT_Security(
            S0 = self.security.S0,
            u = np.exp(step),
            d = np.exp(-step),
            r = self.security.r*delta
        )
        dto = dts.option(self.strike, timesteps, self.put, self.American)
        
        return dto.prices(0)[0]

# Example usage: discrete time

Create an underlying security with initial price $S(0)=100$ which evolves such that $S(t+1)=2S(t)$ if the price goes up and $S(t+1)=\frac{1}{2}S(t)$ if the price goes down. Assume the per-period interest rate is $r=0.05$.

In [2]:
s = DT_Security(S0=100, u=2, d=1/2, r=0.05)

Show how the price of the security can evolve with time.

In [15]:
for t in range(5):
    print([s.price(t, i) for i in range(t + 1)]) # i = number of price increases

[100.0]
[50.0, 200.0]
[25.0, 100.0, 400.0]
[12.5, 50.0, 200.0, 800.0]
[6.25, 25.0, 100.0, 400.0, 1600.0]


Find the risk-neutral probability of the price going up.

In [16]:
s.risk_neutral_p()

0.3666666666666667

Generate a random price at time $t=4$.

In [17]:
s.random_price(t=4)

100.0

Create an American put option on the underlying security (European call options are the default - to create American and/or put options, set the `American` and/or `put` parameters to `True`).

In [18]:
opt = s.option(strike=150, maturity=4, put=True, American=True)

See how the payoff (if exercised immediately) from the option evolves over time (in the case of a European option, only $t=$ maturity has any real significance).

In [19]:
for t in range(5):
    print([opt.payoff(t, i) for i in range(t + 1)])

[50.0]
[100.0, 0]
[125.0, 50.0, 0]
[137.5, 100.0, 0, 0]
[143.75, 125.0, 50.0, 0, 0]


Show how the option price evolves over time.

In [20]:
for t in range(5): # must not go beyond maturity
    print(opt.prices(t))

[77.54197448974828]
[100.13777409846946, 49.086771686922845]
[125.0, 70.84908037288989, 18.190980095741995]
[137.5, 100.0, 30.158730158730155, 0.0]
[143.75, 125.0, 50.0, 0, 0]


# Example usage: continuous-time

Define a security whose price $S(t)$ evolves according to a geometric Brownian motion process, with initial price $S(0)=105$ and volatility $\sigma=0.3$. Suppose the continuously compounded risk-free interest rate is $r=0.1$.

In [10]:
s = CT_Security(S0=105, volatility=0.3, r=0.1)

Generate a random price (under the risk-neutral drift parameter $\mu$) at $t=1$.

In [60]:
s.random_price(t=1)

123.72362373417968

Find the no-arbitrage price of a European call option on the security with strike price $K=100$ which matures in six months. Show that, as the number of timesteps in the calculation tends to infinity, the calculated price converges to the one given by the Black-Scholes formula
$$
    C=s\Phi \left( \omega \right) -Ke^{-\widehat{r}t}\Phi \left( \omega -\sigma \sqrt{t}\right),\hspace{10mm}
    \omega =\dfrac{\widehat{r}t+\sigma ^{2}t/2-\ln \left( K/s\right) }{\sigma \sqrt{t}}
$$
(observe that the rate of convergence is not particularly fast).

In [11]:
from scipy.stats import norm

def Black_Scholes(s, sigma, K, t, r):
    omega = (r*t + sigma**2*t/2 - np.log(K/s)) / (sigma*np.sqrt(t))
    return s*norm.cdf(omega) - K*np.exp(-r*t)*norm.cdf(omega - sigma*np.sqrt(t))

# 'American' and 'put' parameters default to False
opt = s.option(strike=100, maturity=0.5)

[opt.initial_price(timesteps=n) for n in [1, 10, 100, 1000, 2000]]

[16.017546167272176,
 14.424040733509239,
 14.28521115010528,
 14.288238844241805,
 14.286677625538012]

In [12]:
print(Black_Scholes(s.S0, s.volatility, opt.strike, opt.maturity, s.r))

14.286777583621593


Using the above European call option as an example, verify the Put-call Parity formula
$$
    C-P=S-Ke^{-\hat{r}t}.
$$

In [63]:
C_E = opt.initial_price(timesteps=2000)
P_E = s.option(opt.strike, opt.maturity, put=True).initial_price(timesteps=2000)
S = s.S0
K = opt.strike 
r = s.r
t = opt.maturity

C_E - P_E - S + K*np.exp(-r*t)

-5.9450877827771365e-05

Verify that the prices of European and American call options are equal.

In [64]:
C_A = s.option(
    opt.strike,
    opt.maturity,
    American=True
).initial_price(timesteps=2000)

C_E - C_A

0.0

Find the approximate price of an American put option with the same parameters as above (no formula available). Verify that the price is at least as large as that of an European put option.

In [65]:
P_A = s.option(
    opt.strike,
    opt.maturity,
    put=True,
    American=True
).initial_price(timesteps=2000)

print(P_E)
print(P_A)

4.409679526487245
4.743557724080753
